<a href="https://colab.research.google.com/github/martin-embryo/colab/blob/master/Training_split_wells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datetime import datetime
import sanity_checks

import numpy as np
import cv2
import sys
import csv
import os
import shutil
import glob
import h5py

from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import matplotlib
import hashlib

import argparse
from PIL import Image, ImageFilter, ImageEnhance

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.utils import to_categorical, HDF5Matrix
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2

from google.cloud import storage

project_id    = 'keen-incline-232408'
bucket_name   = 'embryonicsbucket'
model_dir  = 'Upload/Models'
data_dir   = 'Upload/Data'

storage_client = storage.Client(project = project_id)
bucket = storage_client.get_bucket(bucket_name)

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [2]:
#sanity checks
print(datetime.now())
sanity_checks.full()
print bucket.name

2019-05-16 13:40:22.755771
operating system is: Ubuntu 16.04
python version is 2.7.12
tensorflow version is 1.12.2
keras version is 2.2.4
in-built keras version is 2.1.6-tf
available devices: /device:CPU:0, /device:XLA_GPU:0, /device:XLA_CPU:0, /device:GPU:0
embryonicsbucket


In [3]:
def reduce_image(img_np):
    img_np = np.mean(img_np, axis=2)
    img_np = cv2.resize(img_np, (20, 26))
    img_np = np.expand_dims(img_np, axis=0)
    img_np = img_np /255.0
    img_np = np.pad(img_np, ((0,0),(3,3),(6,6)), 'constant', constant_values = 1)
    img_np = np.abs(np.subtract(img_np, 1))
    img_np = img_np.reshape((1, 32, 32, 1))
    return img_np

In [4]:
def extract_clock(image, dimens = (500,500)):
    #image_number = image[475:490, 448:490, :]
    image_number = image[(dimens[0]-25):(dimens[0]-10), (dimens[1]-52):(dimens[1]-10), :]
    return image_number

In [5]:
def clock2number(image_number, model, certainty = True):
    SIZE = 7
    
    #>0 digits
    number = []
    for i in range(3):
            img_np = image_number[:, (SIZE*i):(SIZE*(i+1)), :]
            if (np.sum(img_np) > 240*3):
                dig_np = reduce_image(img_np)
                digit_svm = model.predict(dig_np)
                if certainty:
                    if np.sum(digit_svm > 0.1) == 1:
                        digit = np.argmax(digit_svm)
                    else:
                        digit = 'X'
                else:
                    digit = digit_svm
                number.append(str(digit))
        
    number.append('.')
    
    #<0 digits; after decimal place
    img_np = image_number[:, 25:(25+SIZE), :]
    dig_np = reduce_image(img_np)
    digit_svm = model.predict(dig_np)
    if certainty:
        if np.sum(digit_svm > 0.1) == 1:
            digit = np.argmax(digit_svm)
        else:
            digit = 'X'
    else:
        digit = digit_svm
    number.append(str(digit))
        
    if 'X' in number:
        number_np = 0.0
    else:
        number_np = float(''.join(number))
    return number_np

In [6]:
def image2number(image_number):
    SIZE = 7
    
    number = []
    for i in range(3):
            img_np = image_number[:, (SIZE*i):(SIZE*(i+1)), :]
            if (np.sum(img_np) > 240*3):
                dig_np = reduce_image(img_np)
                digit_dl = model.predict_classes(dig_np)
                dig_np = dig_np.reshape(1,32*32)
                digit_svm = clf.predict(dig_np)
                number.append(str(digit_svm[0]))
                #if (digit_svm != digit_dl):
                #    print (digit_svm, digit_dl)
        
    number.append('.')
    
    img_np = image_number[:, (25+SIZE*(4-4)):(25+SIZE*(4-3)), :]
    dig_np = reduce_image(img_np)
    digit_dl = model.predict_classes(dig_np)
    dig_np = dig_np.reshape(1,32*32)
    digit_svm = clf.predict(dig_np)
    number.append(str(digit_svm[0]))
    #if (digit_svm != digit_dl):
    #    print (digit_svm, digit_dl)
        
    number_np = float(''.join(number))
    return number_np

In [7]:
def confirm_movie(blob):
    if '.avi' in blob.name:
        return True
    else:
        return False

In [8]:
def process_video_file(video_names, bucket, output_dir, temp_dir = '/tmp/', fault_dir = './Fault/', \
                       height = 500, width = 500, overwrite_model = False, fault_save = True, \
                       digit_model_name = 'Upload/Models/Digits/model_lenet_embryo.h5', digit_weights_name = 'Upload/Models/Digits/model_lenet_embryo_weights.h5'):
       
    wells_key = {(1,500,500) : ((1,1),(500,500)), (2, 500, 1000): ((1,2),(500,500)), (3,500,1500) : ((1,3),(500,500)), 
             (4, 500, 500): ((2,2),(250,250)), (5, 500, 750): ((2,3),(250,250)), (6,500,750) : ((2,3),(250,250)), \
             (7, 750, 750): ((3,3),(250,250)), (8,750,750) : ((3,3),(250,250)), (9,750,750) : ((3,3),(250,250)), \
             (10,750,1000) : ((3,4),(250,250)), (11, 750, 1000): ((3,4),(250,250)), (12,750,1000) : ((3,4),(250,250))}
    
    digit_model = digit_model_name.split('/')[-1]
    if (not os.path.isfile(temp_dir + digit_model)) or overwrite_model:
        digits_model_gcs = bucket.get_blob(digit_model)
        digits_model_gcs.download_to_filename(temp_dir + digit_model)
    
    digit_weights = digit_weights_name.split('/')[-1]
    if (not os.path.isfile(temp_dir + digit_weights)) or overwrite_model:
        digits_weights_gcs = bucket.get_blob(digit_weights)
        digits_weights_gcs.download_to_filename(temp_dir + digit_weights)
        
    model = load_model(temp_dir + digit_model)
    model.load_weights(temp_dir + digit_weights)
    
    for video_fullname in video_names:
        name = video_fullname.split('/')[-1]
        video_gcs = bucket.get_blob(video_fullname)
        video_gcs.download_to_filename(temp_dir + name)
        video_name = temp_dir + name
        
        wells = map(int, name.split('wells_')[-1].split('_video.')[0].split('_'))
        n_wells = len(wells)
        vidcap = cv2.VideoCapture(video_name)
        length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        success,image = vidcap.read()
        if success and confirm_movie(video_gcs):
            count = 0
            length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
            partition, dimensions = wells_key[(len(wells),image.shape[0],image.shape[1])]
            print video_name, length, image.shape
            print partition, dimensions        
        while success:
            #cv2 is height x width
            #window_size = np.sqrt((image.shape[0]*image.shape[1]) / n_wells)

            for i in range(partition[0]): #rows
                for j in range(partition[1]): #cols
                    if (i*partition[1]+j) < len(wells):
                        crop_img = image[(i*dimensions[0]):((i+1)*dimensions[0]), (j*dimensions[1]):((j+1)*dimensions[1])]
                        filename = name.split('wells_')[0]
                        well = wells[i*partition[1]+j]
                        base_dir = "{}/{}_well{}".format(output_dir, filename, well)
                        if not os.path.exists(base_dir):
                            os.makedirs(base_dir)
                        image_number = extract_clock(crop_img, dimensions)
                        image_number_img = Image.fromarray(image_number)
                        image_number = np.asarray(image_number_img)
                        number = clock2number(image_number, model)
                        if number == 0.0:
                            img = Image.fromarray(image_number)
                            img.save("{}/{}_well{}_f{:03d}_{}h_number.jpg".format(fault_dir, filename, well, count, number))
                        crop_img = cv2.resize(crop_img, (height,width))
                        img = Image.fromarray(crop_img)
                        img = img.crop((0, 0, 474, 447))
                        img = img.resize((height, width))
                        if not os.path.exists("{}/{}_well{}_f{:03d}_{}h.jpg".format(base_dir, filename, well, count, number)):
                            img.save("{}/{}_well{}_f{:03d}_{}h.jpg".format(base_dir, filename, well, count, number))
                        if number == 0.0:
                            img.save("{}/{}_well{}_f{:03d}_{}h.jpg".format(fault_dir, filename, well, count, number))
                        img.close()

            count +=1
            success,image = vidcap.read()

In [ ]:
#Input (without features or labels): bucket link, file list 

#Videos-cloud -> Videos-local -> parsed_frames -> features -> dataset -> experimental dataset 

#Multi-wells, ocr, frame splitting 

#Labeling, patient, splits 
 
#Rotation, 3-window-shifts 

In [ ]:
#mport multiprocessing
#mport itertools
#pool = multiprocessing.Pool(processes = 2)
#process_video_file_args = process_video_file(video_names, bucket = bucket, model_dir = model_dir, data_dir = video_dir, output_dir = './Frames/Ukraine-multi_well_new/')
#pool.map(process_video_file_args, itertools.izip(a_args, itertools.repeat(second_arg)))
#pool.close()
#pool.join()

In [10]:
video_dir = 'Files/Videos/Ukraine/Nadiya_part3/Nadiya_part3/2'

video_blobs = bucket.list_blobs(prefix = video_dir)
video_names = [blob.name for blob in video_blobs]

process_video_file(video_names, bucket = bucket, output_dir = './Frames/Ukraine/')

#process_video_file(video_names, bucket, output_dir, temp_dir = '/tmp/', fault_dir = './Fault/', \
#                       height = 500, width = 500, overwrite_model = False, fault_save = True, \
#                       digit_model_name = 'Upload/Models/Digits/model_lenet_embryo.h5', digit_weights_name = 'Upload/Models/Digits/model_lenet_embryo_weights.h5')

/tmp/D2014.07.29_S0607_I162_wells_1_2_video.avi 387 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2014.10.18_S0671_I162_wells_1_2_video.avi 480 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2015.03.07_S0768_I162_wells_1_2_video.avi 381 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2015.03.17_S0775_I162_wells_1_2_video.avi 579 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2015.03.17_S0776_I162_wells_1_2_video.avi 405 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2015.03.28_S0788_I162_wells_1_2_video.avi 376 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2015.09.03_S0892_I162_wells_1_2_video.avi 394 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2016.03.10_S1032_I162_wells_1_2_video.avi 394 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2016.06.11_S1102_I162_wells_1_2_video.avi 377 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2016.09.07_S1169_I162_wells_1_2_video.avi 472 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2016.10.27_S1212_I162_wells_1_2_video.avi 402 (500, 1000, 3)
(1, 2) (500, 500)
/tmp/D2017.08.11_S01461_I0162_D_wells_1_2_video.avi 469 (500, 100